In [1]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
import numpy as np
import cv2
import math
import os

# 行识别:
# DBNet行检测模型
detect_dbnet_model='damo/cv_resnet18_ocr-detection-db-line-level_damo'
# Seglink 行检测模型 模型依赖tensorflow1.15.x运行 
detect_seglink_model='damo/cv_resnet18_ocr-detection-db-line-level_damo'
# 行识别模型
# 手写:
recognize_hand_model='damo/cv_convnextTiny_ocr-recognition-handwritten_damo'
# 文档
recognize_doc_model='damo/cv_convnextTiny_ocr-recognition-document_damo'
# 自然场景文本
recognize_nature_model='damo/cv_convnextTiny_ocr-recognition-scene_damo'
# 车牌
recognize_plate_model='damo/cv_convnextTiny_ocr-recognition-licenseplate_damo'
# 通用
recognize_general_model='damo/cv_convnextTiny_ocr-recognition-general_damo'

2023-11-24 11:47:16,253 - modelscope - INFO - PyTorch version 2.0.1+cpu Found.
2023-11-24 11:47:16,255 - modelscope - INFO - TensorFlow version 2.13.0 Found.
2023-11-24 11:47:16,255 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-11-24 11:47:16,289 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 79827826d04c54fc06982662c5095533 and a total number of 945 components indexed
/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 选择模型函数:
def mode_func(DBnet, scene, scenes):
    """
    DBnet: 使用DBnet行识别,还是Seglink行识别 (Seglink模型依赖tensorflow1.15.x运行)
    scene: 从列表['handwritings','documents','nature_scene','number_plate','general']选
    scenes: 列表['handwritings','documents','nature_scene','number_plate','general']
    """
    
    # 选择 行检测模型:
    if DBnet: #DBnet
        detect_model = detect_dbnet_model
    else: # Seglink 模型依赖tensorflow1.15.x运行
        detect_model = detect_seglink_model
    
    # 选择 行识别模型:
    if scene == scenes[0] or scene == 0:
        recognize_model = recognize_hand_model
    elif scene == scenes[1] or scene == 1:
        recognize_model = recognize_doc_model
    elif scene == scenes[2] or scene == 2:
        recognize_model = recognize_nature_model
    elif scene == scenes[3] or scene == 3:
        recognize_model = recognize_plate_model
    else:
        recognize_model = recognize_general_model
        
    return detect_model, recognize_model

# scripts for crop images
def crop_image(img, position):
    def distance(x1,y1,x2,y2):
        return math.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))    
    position = position.tolist()
    for i in range(4):
        for j in range(i+1, 4):
            if(position[i][0] > position[j][0]):
                tmp = position[j]
                position[j] = position[i]
                position[i] = tmp
    if position[0][1] > position[1][1]:
        tmp = position[0]
        position[0] = position[1]
        position[1] = tmp

    if position[2][1] > position[3][1]:
        tmp = position[2]
        position[2] = position[3]
        position[3] = tmp

    x1, y1 = position[0][0], position[0][1]
    x2, y2 = position[2][0], position[2][1]
    x3, y3 = position[3][0], position[3][1]
    x4, y4 = position[1][0], position[1][1]

    corners = np.zeros((4,2), np.float32)
    corners[0] = [x1, y1]
    corners[1] = [x2, y2]
    corners[2] = [x4, y4]
    corners[3] = [x3, y3]

    img_width = distance((x1+x4)/2, (y1+y4)/2, (x2+x3)/2, (y2+y3)/2)
    img_height = distance((x1+x2)/2, (y1+y2)/2, (x4+x3)/2, (y4+y3)/2)

    corners_trans = np.zeros((4,2), np.float32)
    corners_trans[0] = [0, 0]
    corners_trans[1] = [img_width - 1, 0]
    corners_trans[2] = [0, img_height - 1]
    corners_trans[3] = [img_width - 1, img_height - 1]

    transform = cv2.getPerspectiveTransform(corners, corners_trans)
    dst = cv2.warpPerspective(img, transform, (int(img_width), int(img_height)))
    return dst

def order_point(coor):
    arr = np.array(coor).reshape([4, 2])
    sum_ = np.sum(arr, 0)
    centroid = sum_ / arr.shape[0]
    theta = np.arctan2(arr[:, 1] - centroid[1], arr[:, 0] - centroid[0])
    sort_points = arr[np.argsort(theta)]
    sort_points = sort_points.reshape([4, -1])
    if sort_points[0][0] > centroid[0]:
        sort_points = np.concatenate([sort_points[3:], sort_points[:3]])
    sort_points = sort_points.reshape([4, 2]).astype('float32')
    return sort_points

In [3]:
img_path_list = ['Letter of intent_页面_1.jpg','Letter of intent_页面_2.jpg','Letter of intent_页面_3.jpg']
txt_out_path = "Letter of intent.txt"

DBnet = True
scenes = ['handwritings','documents','nature_scene','number_plate','general']
scene = 1

detect_model, recognize_model = mode_func(DBnet, scene, scenes)
# 串联文本检测和文本识别模型，以实现完整的OCR功能，
ocr_detection = pipeline(Tasks.ocr_detection, model=detect_model)
ocr_recognition = pipeline(Tasks.ocr_recognition, model=recognize_model)

2023-11-24 11:47:31,708 - modelscope - WARNING - Model revision not specified, use revision: v1.3.0
2023-11-24 11:47:32.878498: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-24 11:47:32.880184: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-24 11:47:32.922719: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-24 11:47:32.923649: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
20

In [ ]:
%%time

# 串联文本检测和文本识别模型，以实现完整的OCR功能，
# 输出的文本列表 txt 
txt_list = []

for img_path in img_path_list:
    if os.path.exists(img_path):
        image_full = cv2.imread(img_path)
        det_result = ocr_detection(image_full)
        det_result = det_result['polygons'] 
        for i in range(det_result.shape[0]):
            pts = order_point(det_result[i]) 
            image_crop = crop_image(image_full, pts) # 获得切割后的文字行
            result = ocr_recognition(image_crop)
            # print("box: %s" % ','.join([str(e) for e in list(pts.reshape(-1))]))
            print(f"{img_path}/line{i}: {result['text']}")
            line_txt = result['text'] # 输出是列表
            txt_list += line_txt
    else:
        print(f"{img_path},文件不存在.")

# 输出文本至文件:
if DBnet:
    txt_list = txt_list[::-1] #倒序,因为DBnet是倒序的
with open(txt_out_path, 'w') as file:
    for text in txt_list: 
        file.write(text + '\n')

Letter of intent_页面_1.jpg/line0: ['m']
Letter of intent_页面_1.jpg/line1: ['3. The target products will use local branding.']
Letter of intent_页面_1.jpg/line2: ['frequency band.']
Letter of intent_页面_1.jpg/line3: ['products in the frequency band used in the Russian market including the 160MHz']
Letter of intent_页面_1.jpg/line4: ['2. The target products will be the wireless trunking system equipment and handset']
Letter of intent_页面_1.jpg/line5: ['under a licensing agreement to be agreed upon by the Parties.']
Letter of intent_页面_1.jpg/line6: ['maintenance of the wireless trunking system and handset products to JSC MRZ']
Letter of intent_页面_1.jpg/line7: ['1. Eastcom will license the know-how for the manufacturing, testing, servicing and']
Letter of intent_页面_1.jpg/line8: ['contained below, the Parties hereto agree as follows:']
Letter of intent_页面_1.jpg/line9: ['NOW, THEREFORE, in consideration of the mutual premises and the covenants']
Letter of intent_页面_1.jpg/line10: ['equipment and hand